In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove-emb/glove.6B.100d.txt
/kaggle/input/fake-news/train.tsv
/kaggle/input/fake-news/valid.tsv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

import nltk
import re
import string 

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing import text,sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
import tensorflow as tf
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant
import re

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_data = pd.read_csv('../input/fake-news/train.tsv',sep="\t")

In [4]:
train_data.columns=['id','label','statement','subjects','speaker','speaker_job_title','state_info','party_affiliation','count_1','count_2','count_3','count_4','count_5','context']

In [5]:
for i in range(0,len(train_data)):
  if((train_data['label'][i]=='true')):
    train_data['label'][i]=1
  else:
    train_data['label'][i]=0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
data=train_data
data.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affiliation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,0,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,0,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,0,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,1,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [7]:
data['text']= data['statement'] + " " + data['subjects'] + " " + data['context']
del data['statement']
del data['id']
del data['subjects']
del data['context']
del data['speaker']
del data['speaker_job_title']
del data['state_info']
del data['party_affiliation']
del data['count_1']
del data['count_2']
del data['count_3']
del data['count_4']
del data['count_5']
data.head()

,label,text
0,0,When did the decline of coal start? It started...
1,0,"Hillary Clinton agrees with John McCain ""by vo..."
2,0,Health care reform legislation is likely to ma...
3,0,The economic turnaround started at the end of ...
4,1,The Chicago Bears have had more starting quart...


In [8]:
data.dropna()
data.fillna("", inplace=True)

In [9]:
pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=fbbeae09c20245b757619d09187edf8f016b17ae57d383ee4a9a6436894321c5
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [10]:
import nltk
nltk.download('punkt')
from bs4 import BeautifulSoup
nltk.download("stopwords")   
from nltk.corpus import stopwords
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
def remove_characters(text):
    return re.sub("[^a-zA-Z]"," ",text)

#Removal of stopwords 
def remove_stopwords_and_lemmatization(text):
    final_text = []
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    for word in text:
        if word not in set(stopwords.words('english')):
            lemma = nltk.WordNetLemmatizer()
            word = lemma.lemmatize(word) 
            final_text.append(word)
    return " ".join(final_text)

#Total function
def cleaning(text):
    text = remove_characters(text)
    text = remove_stopwords_and_lemmatization(text)
    return text

#Apply function on text column
data['text']=data['text'].apply(cleaning)

In [12]:
data=data.sample(frac=1)

**Training when data is 5% labled**

In [13]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.2 ,random_state=42)

In [14]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.95,random_state=0)

In [15]:
max_features = 3000
maxlen = 50

In [16]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [17]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [18]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [19]:
batch_size = 128

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units=160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units , return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate =.001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [21]:
model=build_model()

2022-04-06 12:44:00.695404: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-06 12:44:00.855418: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-06 12:44:00.856259: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-06 12:44:00.857630: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [22]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

2022-04-06 12:44:03.496355: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/300
3/3 [==============================] - 6s 622ms/step - loss: 0.6864 - accuracy: 0.6154 - val_loss: 0.6080 - val_accuracy: 0.8862
Epoch 2/300
3/3 [==============================] - 1s 409ms/step - loss: 0.5860 - accuracy: 0.8182 - val_loss: 0.3657 - val_accuracy: 0.8862
Epoch 3/300
3/3 [==============================] - 1s 454ms/step - loss: 0.4825 - accuracy: 0.8182 - val_loss: 0.3814 - val_accuracy: 0.8862
Epoch 4/300
3/3 [==============================] - 1s 427ms/step - loss: 0.4835 - accuracy: 0.8182 - val_loss: 0.3748 - val_accuracy: 0.8862
Epoch 5/300
3/3 [==============================] - 1s 434ms/step - loss: 0.4770 - accuracy: 0.8182 - val_loss: 0.4024 - val_accuracy: 0.8862
Epoch 6/300
3/3 [==============================] - 2s 395ms/step - loss: 0.4854 - accuracy: 0.8182 - val_loss: 0.3903 - val_accuracy: 0.8862
Epoch 7/300
3/3 [==============================] - 1s 408ms/step - loss: 0.4763 - accuracy: 0.8182 - val_loss: 0.3629 - val_accuracy: 0.8862
Epoch 8/300
3

In [23]:
pred=(model.predict(X_unl) > 0.5).astype("int32")

In [24]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [25]:
model=build_model()

In [26]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
45/45 [==============================] - 24s 439ms/step - loss: 0.2692 - accuracy: 0.9330 - val_loss: 0.1867 - val_accuracy: 0.9557
Epoch 2/100
45/45 [==============================] - 19s 427ms/step - loss: 0.2175 - accuracy: 0.9438 - val_loss: 0.1821 - val_accuracy: 0.9557
Epoch 3/100
45/45 [==============================] - 20s 444ms/step - loss: 0.2165 - accuracy: 0.9438 - val_loss: 0.1814 - val_accuracy: 0.9557
Epoch 4/100
45/45 [==============================] - 20s 443ms/step - loss: 0.2154 - accuracy: 0.9438 - val_loss: 0.1807 - val_accuracy: 0.9557
Epoch 5/100
45/45 [==============================] - 19s 423ms/step - loss: 0.2155 - accuracy: 0.9438 - val_loss: 0.1771 - val_accuracy: 0.9557
Epoch 6/100
45/45 [==============================] - 20s 435ms/step - loss: 0.2108 - accuracy: 0.9437 - val_loss: 0.1749 - val_accuracy: 0.9557
Epoch 7/100
45/45 [==============================] - 20s 434ms/step - loss: 0.2064 - accuracy: 0.9438 - val_loss: 0.1777 - val_accuracy:

In [27]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [28]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

1699


In [29]:
accuracy_5_labled=(count/len(test_res))*100

In [30]:
print("Accuracy at 5% labled data",accuracy_5_labled)

Accuracy at 5% labled data 82.958984375


In [31]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.91      1700
        Real       0.00      0.00      0.00       348

    accuracy                           0.83      2048
   macro avg       0.41      0.50      0.45      2048
weighted avg       0.69      0.83      0.75      2048



**Traning when data 5% labled and lr=0.0001**

In [32]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.2 ,random_state=0)

In [33]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.95,random_state=0)

In [34]:
max_features = 3000
maxlen = 50

In [35]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [36]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [37]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [38]:
batch_size = 128

In [39]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units = 160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units, return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate =.0001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [40]:
model=build_model()

In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           300000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50, 160)           167040    
_________________________________________________________________
lstm_5 (LSTM)                (None, 160)               205440    
_________________________________________________________________
dense_4 (Dense)              (None, 160)               25760     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 161       
Total params: 698,401
Trainable params: 398,401
Non-trainable params: 300,000
_________________________________________________________________


In [42]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
3/3 [==============================] - 5s 607ms/step - loss: 0.6928 - accuracy: 0.5420 - val_loss: 0.6897 - val_accuracy: 0.8537
Epoch 2/300
3/3 [==============================] - 1s 399ms/step - loss: 0.6890 - accuracy: 0.8462 - val_loss: 0.6854 - val_accuracy: 0.8537
Epoch 3/300
3/3 [==============================] - 1s 403ms/step - loss: 0.6849 - accuracy: 0.8497 - val_loss: 0.6803 - val_accuracy: 0.8537
Epoch 4/300
3/3 [==============================] - 1s 416ms/step - loss: 0.6796 - accuracy: 0.8497 - val_loss: 0.6737 - val_accuracy: 0.8537
Epoch 5/300
3/3 [==============================] - 1s 416ms/step - loss: 0.6733 - accuracy: 0.8497 - val_loss: 0.6646 - val_accuracy: 0.8537
Epoch 6/300
3/3 [==============================] - 1s 426ms/step - loss: 0.6644 - accuracy: 0.8497 - val_loss: 0.6520 - val_accuracy: 0.8537
Epoch 7/300
3/3 [==============================] - 2s 562ms/step - loss: 0.6519 - accuracy: 0.8497 - val_loss: 0.6336 - val_accuracy: 0.8537
Epoch 8/300
3

In [43]:
pred=(model.predict(X_unl) > 0.5).astype("int32")

In [44]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [45]:
model=build_model()

In [46]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
45/45 [==============================] - 23s 434ms/step - loss: 0.4570 - accuracy: 0.9676 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 2/100
45/45 [==============================] - 19s 420ms/step - loss: 0.0630 - accuracy: 0.9894 - val_loss: 0.0106 - val_accuracy: 1.0000
Epoch 3/100
45/45 [==============================] - 20s 433ms/step - loss: 0.0589 - accuracy: 0.9894 - val_loss: 0.0087 - val_accuracy: 1.0000
Epoch 4/100
45/45 [==============================] - 20s 434ms/step - loss: 0.0584 - accuracy: 0.9894 - val_loss: 0.0080 - val_accuracy: 1.0000
Epoch 5/100
45/45 [==============================] - 19s 427ms/step - loss: 0.0593 - accuracy: 0.9894 - val_loss: 0.0100 - val_accuracy: 1.0000
Epoch 6/100
45/45 [==============================] - 19s 432ms/step - loss: 0.0592 - accuracy: 0.9894 - val_loss: 0.0092 - val_accuracy: 1.0000
Epoch 7/100
45/45 [==============================] - 19s 423ms/step - loss: 0.0588 - accuracy: 0.9894 - val_loss: 0.0080 - val_accuracy:

In [47]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [48]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

1701


In [49]:
accuracy_50001_labled=(count/len(test_res))*100

In [50]:
print("Accuracy at 5% labled data and lr=.0001",accuracy_50001_labled)

Accuracy at 5% labled data and lr=.0001 83.056640625


In [51]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.91      1701
        Real       0.00      0.00      0.00       347

    accuracy                           0.83      2048
   macro avg       0.42      0.50      0.45      2048
weighted avg       0.69      0.83      0.75      2048



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Training Model when data is 5% labeled and batch size 256**

In [52]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.2 ,random_state=0)

In [53]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.95,random_state=0)

In [54]:
max_features = 3000
maxlen = 50

In [55]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [56]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [57]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [58]:
batch_size = 256

In [59]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units = 160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units, return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate = .001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [60]:
model=build_model()

In [61]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 100)           300000    
_________________________________________________________________
lstm_8 (LSTM)                (None, 50, 160)           167040    
_________________________________________________________________
lstm_9 (LSTM)                (None, 160)               205440    
_________________________________________________________________
dense_8 (Dense)              (None, 160)               25760     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 161       
Total params: 698,401
Trainable params: 398,401
Non-trainable params: 300,000
_________________________________________________________________


In [62]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
2/2 [==============================] - 6s 921ms/step - loss: 0.6919 - accuracy: 0.5979 - val_loss: 0.6624 - val_accuracy: 0.8537
Epoch 2/300
2/2 [==============================] - 1s 465ms/step - loss: 0.6581 - accuracy: 0.8497 - val_loss: 0.5584 - val_accuracy: 0.8537
Epoch 3/300
2/2 [==============================] - 1s 889ms/step - loss: 0.5502 - accuracy: 0.8497 - val_loss: 0.4701 - val_accuracy: 0.8537
Epoch 4/300
2/2 [==============================] - 1s 442ms/step - loss: 0.4767 - accuracy: 0.8497 - val_loss: 0.4668 - val_accuracy: 0.8537
Epoch 5/300
2/2 [==============================] - 1s 438ms/step - loss: 0.4694 - accuracy: 0.8497 - val_loss: 0.4173 - val_accuracy: 0.8537
Epoch 6/300
2/2 [==============================] - 1s 440ms/step - loss: 0.4286 - accuracy: 0.8497 - val_loss: 0.4607 - val_accuracy: 0.8537
Epoch 7/300
2/2 [==============================] - 1s 468ms/step - loss: 0.4696 - accuracy: 0.8497 - val_loss: 0.4634 - val_accuracy: 0.8537
Epoch 8/300
2

In [63]:
pred=(model.predict(X_unl) > 0.5).astype("int32")

In [64]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [65]:
model=build_model()

In [66]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
23/23 [==============================] - 14s 473ms/step - loss: 0.3162 - accuracy: 0.9365 - val_loss: 0.2207 - val_accuracy: 0.9439
Epoch 2/100
23/23 [==============================] - 10s 427ms/step - loss: 0.2216 - accuracy: 0.9426 - val_loss: 0.2185 - val_accuracy: 0.9439
Epoch 3/100
23/23 [==============================] - 10s 418ms/step - loss: 0.2214 - accuracy: 0.9426 - val_loss: 0.2171 - val_accuracy: 0.9439
Epoch 4/100
23/23 [==============================] - 11s 473ms/step - loss: 0.2203 - accuracy: 0.9426 - val_loss: 0.2164 - val_accuracy: 0.9439
Epoch 5/100
23/23 [==============================] - 10s 424ms/step - loss: 0.2201 - accuracy: 0.9426 - val_loss: 0.2160 - val_accuracy: 0.9439
Epoch 6/100
23/23 [==============================] - 10s 431ms/step - loss: 0.2210 - accuracy: 0.9426 - val_loss: 0.2160 - val_accuracy: 0.9439
Epoch 7/100
23/23 [==============================] - 10s 456ms/step - loss: 0.2218 - accuracy: 0.9426 - val_loss: 0.2171 - val_accuracy:

In [67]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [68]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

1698


In [69]:
accuracy_5_bs_256_labled=(count/len(test_res))*100

In [70]:
print("Accuracy at 5% labled data and bs 256",accuracy_5_bs_256_labled)

Accuracy at 5% labled data and bs 256 82.91015625


In [71]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.91      1701
        Real       0.33      0.01      0.02       347

    accuracy                           0.83      2048
   macro avg       0.58      0.50      0.46      2048
weighted avg       0.75      0.83      0.76      2048



**Training Model when 5% labled data and bs=512**

In [72]:
X_train1, X_test, y_train1, y_test = train_test_split(data['text'], data['label'],test_size=0.2 ,random_state=42)

In [73]:
X_lvl, X_unl, y_lvl, y_unl = train_test_split(X_train1,y_train1,test_size=0.95,random_state=0)

In [74]:
max_features = 3000
maxlen = 50

In [75]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(X_lvl)
tokenized_train = tokenizer.texts_to_sequences(X_lvl)
X_lvl = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [76]:
tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [77]:
tokenized_unl = tokenizer.texts_to_sequences(X_unl)
X_unl = sequence.pad_sequences(tokenized_unl, maxlen=maxlen)

In [78]:
batch_size = 512

In [79]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers

def build_model():
  model = Sequential()
  hp_units = 160
  model.add(Embedding(max_features, output_dim=100, input_length=maxlen, trainable=False))
  model.add(LSTM(units=hp_units, return_sequences = True , recurrent_dropout =0.5,dropout=0.5))
  model.add(LSTM(units=hp_units , recurrent_dropout = 0.5 , dropout = 0.5))
  model.add(Dense(units=hp_units , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  hp_learning_rate = .001
  model.compile(tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [80]:
model = build_model()

In [81]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 50, 100)           300000    
_________________________________________________________________
lstm_12 (LSTM)               (None, 50, 160)           167040    
_________________________________________________________________
lstm_13 (LSTM)               (None, 160)               205440    
_________________________________________________________________
dense_12 (Dense)             (None, 160)               25760     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 161       
Total params: 698,401
Trainable params: 398,401
Non-trainable params: 300,000
_________________________________________________________________


In [82]:
history = model.fit(X_lvl, y_lvl, validation_split=0.3, epochs=300, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/300
1/1 [==============================] - 4s 4s/step - loss: 0.6937 - accuracy: 0.4091 - val_loss: 0.6826 - val_accuracy: 0.8862
Epoch 2/300
1/1 [==============================] - 0s 438ms/step - loss: 0.6841 - accuracy: 0.8182 - val_loss: 0.6654 - val_accuracy: 0.8862
Epoch 3/300
1/1 [==============================] - 0s 493ms/step - loss: 0.6700 - accuracy: 0.8182 - val_loss: 0.6336 - val_accuracy: 0.8862
Epoch 4/300
1/1 [==============================] - 0s 417ms/step - loss: 0.6438 - accuracy: 0.8182 - val_loss: 0.5701 - val_accuracy: 0.8862
Epoch 5/300
1/1 [==============================] - 0s 423ms/step - loss: 0.5927 - accuracy: 0.8182 - val_loss: 0.4496 - val_accuracy: 0.8862
Epoch 6/300
1/1 [==============================] - 0s 431ms/step - loss: 0.5112 - accuracy: 0.8182 - val_loss: 0.3763 - val_accuracy: 0.8862
Epoch 7/300
1/1 [==============================] - 0s 452ms/step - loss: 0.5355 - accuracy: 0.8182 - val_loss: 0.3669 - val_accuracy: 0.8862
Epoch 8/300
1/1 

In [83]:
pred=(model.predict(X_unl) > 0.50).astype("int32")

In [84]:
X_train=np.concatenate((X_lvl,X_unl))
pred2=pred.flatten()
y_unl = pd.Series(pred2)
y_train=np.concatenate((y_lvl,y_unl))

In [85]:
history = model.fit(X_train, y_train, validation_split=0.3, epochs=100, batch_size=batch_size, shuffle=True, verbose = 1)

Epoch 1/100
12/12 [==============================] - 5s 416ms/step - loss: 0.1448 - accuracy: 0.9583 - val_loss: 0.1048 - val_accuracy: 0.9707
Epoch 2/100
12/12 [==============================] - 5s 438ms/step - loss: 0.1254 - accuracy: 0.9644 - val_loss: 0.0841 - val_accuracy: 0.9707
Epoch 3/100
12/12 [==============================] - 5s 412ms/step - loss: 0.1149 - accuracy: 0.9646 - val_loss: 0.0773 - val_accuracy: 0.9703
Epoch 4/100
12/12 [==============================] - 5s 423ms/step - loss: 0.1144 - accuracy: 0.9660 - val_loss: 0.0752 - val_accuracy: 0.9707
Epoch 5/100
12/12 [==============================] - 5s 434ms/step - loss: 0.1133 - accuracy: 0.9651 - val_loss: 0.0761 - val_accuracy: 0.9711
Epoch 6/100
12/12 [==============================] - 5s 456ms/step - loss: 0.1141 - accuracy: 0.9651 - val_loss: 0.0747 - val_accuracy: 0.9707
Epoch 7/100
12/12 [==============================] - 5s 445ms/step - loss: 0.1106 - accuracy: 0.9655 - val_loss: 0.0763 - val_accuracy: 0.9707

In [86]:
test_res=(model.predict(X_test) > 0.5).astype("int32")
test_res=test_res.flatten()
y_test_c = y_test.to_numpy()

In [87]:
count=0
for i in range(0,len(y_test_c)):
  if(y_test_c[i]==test_res[i]):
    count=count+1
print(count)

1697


In [88]:
accuracy_5_labled_bs_512=(count/len(test_res))*100

In [89]:
print("Accuracy at 5% labled data and bs=512",accuracy_5_labled_bs_512)

Accuracy at 5% labled data and bs=512 82.861328125


In [90]:
print(classification_report(y_test,test_res, target_names = ['Fake','Real']))

              precision    recall  f1-score   support

        Fake       0.83      1.00      0.91      1700
        Real       0.00      0.00      0.00       348

    accuracy                           0.83      2048
   macro avg       0.41      0.50      0.45      2048
weighted avg       0.69      0.83      0.75      2048

